In [ ]:
# from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install keras_tuner

In [1]:
# importing os module
import os
import numpy as np
from PIL import Image
from numpy import asarray
from tensorflow import keras
from tensorflow.keras.preprocessing import image

In [2]:
#Function to convert images to array
def img_to_array(target_folder, img_size):
    #main_folder = "C:/Users/User/Project 4 local/archive/" 
    x = []
    y = []
    z = []
    path = "C:/Users/User/Project 4 local/archive/"
    main_folder = path + target_folder
    for count, foldername in enumerate(os.listdir(main_folder)):
        print(foldername)
        folder = main_folder + "/" + foldername
        for count, filename in enumerate(os.listdir(folder)):
            # load convert and resize the image
            img_path = os.path.join(folder, filename)
            img = image.load_img(img_path, 
                                 target_size=(img_size, img_size, 1), 
                                 color_mode="grayscale")
            #img = Image.open(f"{folder}/{filename}")
            # Resize the image
            #img = img.resize((img_size, img_size))
            #convert the image to grayscale
            #img = img.convert('L')

            # PIL images into NumPy arrays
            img_data = asarray(img)
            x.append(img_data)
            y.append(foldername)
            z.append(target_folder)
    return x,y,z

In [16]:
target_folder1= "human_face"
target_folder2 = "others"
#human_face_path = "C:/Users/User/Project 4 local/archive/human_face/"
img_size = 48
# Call image to array function to convert all Human Face images to array
emotion_feature,emotion_target,mod1target = img_to_array(target_folder1, img_size)
print(len(emotion_feature))
print(len(emotion_target))
print(len(mod1target))

anger
contempt
disgust
fear
happiness
neutrality
sadness
surprise
28992
28992
28992


In [4]:
# Call image to array function to convert all Others images to array
others_feature,others,otherstarget = img_to_array(target_folder2, img_size)
print(len(others_feature))
print(len(others))
print(len(otherstarget))

car
cat
dog
emoji
flower
fruit
wild
7921
7921
7921


## Human Face or Others

In [87]:
# #for Conv2D Model
# features1 = emotion_feature + others_feature
# target1 = mod1target + otherstarget

# features1_arr = np.array(features1).reshape(-1, img_size, img_size, 1)
# target1_arr = np.array(target1)
# print(features1_arr.shape)
# print(target1_arr.shape)

(12119, 48, 48, 1)
(12119,)


In [17]:
#For Face Models only
features1 = others_feature + emotion_feature
target1 = otherstarget + mod1target

features1_arr = np.array(features1)
target1_arr = np.array(target1)
print(features1_arr.shape)
print(target1_arr.shape)

(36913, 48, 48)
(36913,)


In [18]:
# Dependencies to Visualise the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
import tensorflow as tf
# Sklearn
from sklearn.model_selection import train_test_split
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# Keras
from tensorflow import keras
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AveragePooling2D, Conv2DTranspose
from keras.metrics import top_k_categorical_accuracy
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [25]:
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(features1_arr, target1_arr, random_state=43)
X_train.shape

(27684, 48, 48)

In [51]:
# #For Conv2D model
# # building the input vector from the 48x48 pixels
# X_train = X_train.reshape(X_train.shape[0], img_size, img_size,1)
# X_test = X_test.reshape(X_test.shape[0], img_size, img_size,1)
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')

In [26]:
#For Dense models only
# We want to flatten our image of 224x224 pixels to a 1D array of 50176 pixels
ndims = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], ndims)
X_test = X_test.reshape(X_test.shape[0], ndims)
print("Training Shape:", X_train.shape)
print("Testing Shape:", X_test.shape)

Training Shape: (27684, 2304)
Testing Shape: (9229, 2304)


In [ ]:
# #For Conv2D model
# # normalizing the data to help with the training
# X_train /= 255
# X_test /= 255

In [27]:
# Next, we normalise our training data to be between 0 and 1
scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Training Shape:", X_train_scaled.shape)
print("Testing Shape:", X_test_scaled.shape)
X_train.shape[1]

Training Shape: (27684, 2304)
Testing Shape: (9229, 2304)


2304

In [28]:
X_train_scaled.shape

(27684, 2304)

In [29]:
# Change target catagorieslabels to numaeric categories using LabelEncoder
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)
classes = le.classes_
classes

array(['human_face', 'others'], dtype='<U10')

In [30]:
# We need to convert our target labels (expected values) to categorical data
num_classes = len(classes)
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)
# Original label of `Surprise` is one-hot encoded as `00000001`
y_train_cat[0]

array([0., 1.], dtype=float32)

In [24]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','swish'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(Dense(units=hp.Int('first_units',
        min_value=20,
        max_value=200,
        step=10), activation=activation, input_dim= X_train_scaled.shape[1]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(Dense(units=hp.Int('units_' + str(i),
            min_value=20,
            max_value=200,
            step=10),
            activation=activation))
    
    nn_model.add(Dense(units=2, activation='sigmoid'))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [13]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

In [14]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train_cat,epochs=20,validation_data=(X_test_scaled,y_test_cat))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
relu              |?                 |activation
100               |?                 |first_units
2                 |?                 |num_layers
100               |?                 |units_0
3                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/3
216/889 [======>.......................] - ETA: 32s - loss: 0.4127 - accuracy: 0.8086

KeyboardInterrupt: 

In [79]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'swish',
 'first_units': 120,
 'num_layers': 2,
 'units_0': 90,
 'units_1': 100,
 'units_2': 120,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test_cat,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [31]:
model = Sequential()

activation = 'swish'

    
# first layer with input dimention
model.add(Dense(120, activation=activation, 
                input_dim= X_train_scaled.shape[1]))

# Hidden layers
#model.add(Dense(90, activation=activation))
model.add(Dense(100, activation=activation))
# Output layer
model.add(Dense(2, activation='sigmoid'))

# Compile the model
model.compile(loss="binary_crossentropy", optimizer='adam', 
              metrics=["accuracy"])

In [32]:
# We can summarise our model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 120)               276600    
                                                                 
 dense_5 (Dense)             (None, 100)               12100     
                                                                 
 dense_6 (Dense)             (None, 2)                 202       
                                                                 
Total params: 288,902
Trainable params: 288,902
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_cat,
    epochs=2,
    shuffle=True,
    verbose=2,
    validation_data=(X_test_scaled, y_test_cat)
)

Epoch 1/2
866/866 - 64s - loss: 0.3196 - accuracy: 0.8554 - val_loss: 0.2924 - val_accuracy: 0.8853 - 64s/epoch - 74ms/step
Epoch 2/2
866/866 - 98s - loss: 0.2604 - accuracy: 0.8867 - val_loss: 0.2800 - val_accuracy: 0.8816 - 98s/epoch - 113ms/step


In [ ]:
#  "Accuracy"
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# "Loss"
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Saving and Loading models

We can save our trained models using the HDF5 binary format with the extension `.h5`

In [15]:
# Save the model
model.save("trained_models/face_model.h5")

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("emotion_trained.h5")

## Evaluating the Model

We use our testing data to validate our model. This is how we determine the validity of our model (i.e. the ability to predict new and previously unseen data points)

In [ ]:
# Evaluate the model using the training data 
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_cat, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

## Making Predictions

We can use our trained model to make predictions using `model.predict`

In [ ]:
# Define default image size for our model
image_path = os.path.join("Images", "happy.jpeg")
image_size = (48, 48)

In [ ]:
from tensorflow.keras.preprocessing import image
im = image.load_img(image_path, target_size=image_size, color_mode="grayscale")
im

In [ ]:
# Convert the image to a numpy array 
from tensorflow.keras.preprocessing.image import img_to_array
image = img_to_array(im)

# Scale the image pixels by 255 (or use a scaler from sklearn here)
image /= 255

# Flatten into a 1x28*28 array 
img = image.flatten().reshape(-1, 224*224)
img.shape


In [ ]:
# Invert the pixel values to match the original data
img = 1 - img
plt.imshow(img.reshape(224, 224), cmap=plt.cm.Greys)

In [ ]:
# Make a prediction. The result should be 0000010000000 for a 5
model.predict(img).round()